In [1]:
from general_tools.notebook import gpu_utils
GPU = 3
gpu_utils.setup_one_gpu(GPU)

Picking GPU 3


In [2]:
import sys
import time
import numpy as np
import os.path as osp
import tensorflow as tf
import matplotlib.pyplot as plt
from scipy.stats import hmean
from sklearn.manifold import TSNE

from general_tools.notebook.tf import reset_tf_graph

import tf_lab.point_clouds.in_out as pio
from tf_lab.point_clouds.in_out import PointCloudDataSet, write_model_ids_of_datasets
from tf_lab.point_clouds.point_net_ae import PointNetAutoEncoder
from tf_lab.point_clouds.autoencoder import Configuration as Conf
import tf_lab.point_clouds.encoders_decoders as enc_dec

from tf_lab.autopredictors.scripts.helper import shape_net_category_to_synth_id, points_extension, \
                                                 shape_net_core_synth_id_to_category

from tf_lab.autopredictors.plotting import plot_original_pclouds_vs_reconstructed, \
                                           plot_train_val_test_curves, plot_reconstructions_at_epoch
        
from tf_lab.autopredictors.evaluate import eval_model, read_saved_epochs, accuracy_of_completion, \
                                           coverage_of_completion, save_reconstructions, \
                                           save_pc_prediction_stats, save_stats_of_multi_class_experiments, \
                                           paper_pc_completion_experiment_id_best_epoch
                                                  
from tf_lab.autopredictors.exploration import latent_embedding_of_entire_dataset, \
                                              embedding_of_entire_dataset_at_tensor

from general_tools.in_out.basics import create_dir, delete_files_in_directory, files_in_subdirs
from general_tools.simpletons import select_first_last_and_k
from geo_tool import Point_Cloud

from tf_lab.nips.data_sets.model_net import pc_loader, classes_to_integers
from tf_lab.nips.helper import center_pclouds_in_unit_sphere, average_per_class, zero_mean_half_sphere

from tf_lab.point_clouds.encoders_decoders import decoder_with_fc_only
from tf_lab.point_clouds.encoders_decoders import encoder_with_convs_and_symmetry
from sklearn.svm import LinearSVC

/usr/local/lib/python2.7/dist-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [3]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [12]:
n_pc_samples = 2048
put_in_usphere = True
model_net = '10'

# experiment_name ='all_snc_rotated_conv_arch_2048pts_chamfer'
# experiment_name ='all_snc_rotated_conv_arch_2048pts_emd'

# experiment_name ='wu_classes_rotated_convolutional_arch_2048pts_chamfer'
experiment_name ='wu_classes_rotated_convolutional_arch_2048pts_emd'

In [13]:
top_data_dir = '/orions4-zfs/projects/lins2/Panos_Space/DATA/'
train_dir = osp.join(top_data_dir, 'OUT/models/nips/vanilla_ae/')
train_dir = osp.join(train_dir, experiment_name)

conf = Conf.load(osp.join(train_dir, 'configuration'))
print conf
conf.n_output = conf.n_input

                    batch_size: 50
                 consistent_io: None
                         debug: False
                       decoder: decoder_with_fc_only
                  decoder_args: {'layer_sizes': [1024, 2048, 6144]}
                       encoder: encoder_with_convs_and_symmetry
                  encoder_args: {'filter_sizes': [40, 20, 10, 10], 'n_filters': [128, 128, 256, 512], 'strides': [1, 2, 2, 1]}
               experiment_name: wu_classes_rotated_convolutional_arch_2048pts_emd
                 gauss_augment: None
                  is_denoising: False
               latent_vs_recon: 1.0
                 learning_rate: 0.0005
                          loss: emd
             loss_display_step: 1
                       n_input: [2048, 3]
                      n_output: [2048, 3]
                           n_z: None
             saver_max_to_keep: None
                    saver_step: 10
                     train_dir: /orions4-zfs/projects/lins2/Panos_Space/DATA/OUT/mo

In [6]:
reset_tf_graph()
ae = PointNetAutoEncoder(experiment_name, conf)
saved_epochs = read_saved_epochs(conf.train_dir)
last_epoch = saved_epochs[-1]
# last_epoch = 2000
ae.restore_model(conf.train_dir, last_epoch, verbose=True)

Model restored in epoch 2640.


In [14]:
model_net_dir = '/orions4-zfs/projects/lins2/Panos_Space/DATA/Point_Clouds/Model_Net_' + model_net +  '/from_manifold/' 
model_net_dir = osp.join(model_net_dir, str(n_pc_samples))

search_pattern = '(.*)train(.*)\.ply$'
train_pc_files = [f for f in files_in_subdirs(model_net_dir, search_pattern)]

search_pattern = '(.*)test(.*)\.ply$'
test_pc_files = [f for f in files_in_subdirs(model_net_dir, search_pattern)]

In [15]:
pc, model_names, labels = pio.load_crude_point_clouds(train_pc_files, loader=pc_loader, n_threads=10, verbose=True)

if put_in_usphere:
#     pc = center_pclouds_in_unit_sphere(pc)
    pc = zero_mean_half_sphere(pc)
    
train_data = PointCloudDataSet(pc, labels=labels)

pc, model_names, labels = pio.load_crude_point_clouds(test_pc_files, loader=pc_loader, n_threads=10, verbose=True)

if put_in_usphere:
#     pc = center_pclouds_in_unit_sphere(pc)
    pc = zero_mean_half_sphere(pc)
    
test_data = PointCloudDataSet(pc, labels=labels)

3991 pclouds were loaded. They belong in 10 shape-classes.
908 pclouds were loaded. They belong in 10 shape-classes.


In [16]:
train_feed, train_latent, train_classes = latent_embedding_of_entire_dataset(train_data, ae, conf)
cids = classes_to_integers(int(model_net), train_classes)[1]
test_feed, test_latent, test_classes = latent_embedding_of_entire_dataset(test_data, ae, conf)
cids_ = classes_to_integers(int(model_net), test_classes)[1]

In [14]:
lsvc = LinearSVC(C=1, loss='hinge', intercept_scaling=4.0)
lsvc.fit(train_latent, cids)
print lsvc.score(test_latent, cids_)
print average_per_class(lsvc, test_latent, cids_)

0.948237885463
0.94311627907


In [17]:
c_space = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
intercept_scaling_space = [0.5, 1, 2, 3, 4]
dual_space = [False, True]
loss_space = ['hinge', 'squared_hinge']

for c in c_space:
    for intercept in intercept_scaling_space:
        for dual in dual_space:
            for loss in loss_space:
                try:
                    lsvc = LinearSVC(C=c, loss=loss, intercept_scaling=intercept, dual=dual)
                    lsvc.fit(train_latent, cids)
                    print c, intercept, dual, loss
                    print average_per_class(lsvc, test_latent, cids_)                
                except:
                    continue

0.1 0.5 False squared_hinge
0.942465116279
0.1 0.5 True hinge
0.930651162791
0.1 0.5 True squared_hinge
0.942465116279
0.1 1 False squared_hinge
0.942465116279
0.1 1 True hinge
0.929651162791
0.1 1 True squared_hinge
0.942465116279
0.1 2 False squared_hinge
0.943627906977
0.1 2 True hinge
0.930651162791
0.1 2 True squared_hinge
0.943627906977
0.1 3 False squared_hinge
0.943627906977
0.1 3 True hinge
0.931813953488
0.1 3 True squared_hinge
0.943627906977
0.1 4 False squared_hinge
0.944790697674
0.1 4 True hinge
0.932976744186
0.1 4 True squared_hinge
0.944790697674
0.2 0.5 False squared_hinge
0.942302325581
0.2 0.5 True hinge
0.941139534884
0.2 0.5 True squared_hinge
0.942302325581
0.2 1 False squared_hinge
0.942302325581
0.2 1 True hinge
0.942302325581
0.2 1 True squared_hinge
0.942302325581
0.2 2 False squared_hinge
0.943302325581
0.2 2 True hinge
0.941302325581
0.2 2 True squared_hinge
0.943302325581
0.2 3 False squared_hinge
0.945627906977
0.2 3 True hinge
0.939139534884
0.2 3 True 

In [ ]:
# With WU 7 Classes:

# M10, CD, 2K epochs
# 0.4 4 True hinge
# 0.95176744186

# M40, CD, 2K epochs
# 1.0 1 True hinge
# 0.84410495716

# M10, EMD, ~2.5K epochs
# about 0.94

# M40, EMD, ~2.5K epochs
# about 0.83

In [ ]:


# USING ALL ShapeNetCore DATA : ~57K models.
# Training at (max) 2K epochs.

# EMD, 2K, model_net_40
# 1.0 3 False squared_hinge
# 0.844814259486
# 1.0 3 True squared_hinge
# 0.844814259486

# EMD, 2K, model_net_10
# 0.4 2 True hinge
# 0.953279069767


# Chamfer, 2K model_net_40
# 0.3 4 False squared_hinge
# 0.856074051408
# 0.3 4 True squared_hinge
# 0.856074051408
# 0.6 1 False squared_hinge
# 0.856264075887
# 0.3 4 False squared_hinge
# 0.856074051408
# 0.6 1 False squared_hinge
# 0.856264075887
# 0.6 1 True squared_hinge
# 0.856264075887
# 0.7 1 False squared_hinge
# 0.856764075887

# Chamfer, 2K model_net_10
# 0.9 4 True hinge
# 0.950790697674
# 0.950790697674
# 0.8 0.5 False squared_hinge

# 1.0 4 True hinge
# 0.950790697674
# 1.0 3 True hinge
# 0.950790697674
# 0.9 4 True hinge
# 0.950790697674

# 0.7 4 True hinge
# 0.950790697674


